In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)

from models import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split
)

Global seed set to 42
Global seed set to 42


##  Get Data and train a Neural Network

In [2]:
ROOT_PATH = 'c:/Users/gilbe/Documents/TokyoData'


'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
train_df['Date'] = pd.to_datetime(train_df['Date']) 
train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)


import matplotlib.pyplot as plt

df_1301 = train_df[train_df['SecuritiesCode'] == 1301].drop('SecuritiesCode', axis=1)

df_train, df_val = ts_split(df_1301)

xtrain, ytrain = preprocess(df_train)
xval, yval = preprocess(df_val)

### Train the model

In [3]:
import torch
from sklearn.impute import SimpleImputer


imp = SimpleImputer(missing_values=np.nan, strategy='mean')

# xtrain = imp.fit_transform(xtrain)
train_dataloader = get_loader(xtrain, ytrain, 64)
val_dataloader = get_loader(xval, yval, 64)


model = NeuralNetwork(in_features=xtrain.shape[1], out_features=1)
trainer = Trainer(model, lr=3e-6)
trainer.fit_one_epoch(train_dataloader, val_dataloader)


Using cuda-device
loss: 2.9259369373321533 [0/1]
val-loss: 1.5899758338928223 [1/1]
